## Pacchetti

In [2]:
import pandas as pd
import requests
from tqdm import tqdm
import time
from datetime import datetime, timedelta
#from google.colab import drive
#drive.mount('/content/drive/')

/Users/andreamdipalma/DataspellProjects/knowledge2dataset/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Moduli

In [3]:
def normalize_title(title):
    return title.strip().lower().replace(' ', '_')

def process_batch(titles_batch, api_url="https://en.wikipedia.org/w/api.php"):
    params = {
        "action": "query",
        "format": "json",
        "titles": "|".join(titles_batch.keys()),
        "prop": "pageprops"
    }
    try:
        response = requests.get(api_url, params=params)
        response.raise_for_status()
        pages = response.json().get('query', {}).get('pages', {})
        batch_entities = {}
        for page_id, page_info in pages.items():
            pageprops = page_info.get('pageprops', {})
            wikidata_id = pageprops.get('wikibase_item')
            if wikidata_id:
                title = page_info['title']
                normalized_title = normalize_title(title)
                if normalized_title in titles_batch:
                    batch_entities[title] = {
                        "wikidata_id": wikidata_id,
                        "views": titles_batch[normalized_title]
                    }
        return batch_entities
    except requests.RequestException as e:
        print(f"Failed to fetch data: {e}")
        return {}  # Return an empty dict in case of failure

def fetch_wikidata_ids(most_viewed_pages, batch_size=50, api_url="https://en.wikipedia.org/w/api.php"):
    entities = {}
    titles_batch = {}

    for page in most_viewed_pages:
        normalized_title = normalize_title(page['article'])
        titles_batch[normalized_title] = page['views']
        if len(titles_batch) >= batch_size:
            entities.update(process_batch(titles_batch, api_url))
            titles_batch = {}  # Reset the batch

    if titles_batch:  # Process any remaining titles
        entities.update(process_batch(titles_batch, api_url))

    return entities

def get_relations_batch(wikidata_ids, batch_size=50, api_url="https://www.wikidata.org/w/api.php"):
    relations = {}
    for start in range(0, len(wikidata_ids), batch_size):
        end = start + batch_size
        batch = wikidata_ids[start:end]
        wikidata_ids_str = "|".join(batch)

        print(f"Processing batch {start // batch_size + 1} out of {len(wikidata_ids) // batch_size + 1}")

        params = {
            "action": "wbgetentities",
            "format": "json",
            "ids": wikidata_ids_str,
            "props": "claims"
        }

        response = requests.get(api_url, params=params)
        if response.status_code != 200:
            print(f"Error: Received status code {response.status_code}")
            continue

        try:
            data = response.json().get('entities', {})
        except ValueError:
            print("Error: Unable to parse JSON response")
            continue

        for wikidata_id in batch:
            if wikidata_id in data:
                claims = data[wikidata_id].get('claims', {})
                relations[wikidata_id] = claims
            else:
                print(f"Warning: No data found for Wikidata ID {wikidata_id}")

    return relations


def query(date):
    url = f'https://wikimedia.org/api/rest_v1/metrics/pageviews/top/en.wikipedia/all-access/{date}'
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"

    result = requests.get(url, headers={'User-Agent': user_agent}).json()

    if 'error' in result:
        raise Exception(result['error'])
    if 'warnings' in result:
        print(result['warnings'])
    if 'items' in result:
        return result['items'][0]['articles']
    return []

def most_viewed(n):
    unique_articles = {}
    date = datetime.strptime("2024-01-01", "%Y-%m-%d")

    while len(unique_articles) < n:
        formatted_date = date.strftime("%Y/%m/%d")
        articles = query(formatted_date)

        for article in articles:
            unique_articles[article['article']] = article

        date -= timedelta(days=1)  # Passa al giorno precedente

    # Prendi solo i primi n articoli
    most_viewed_articles = list(unique_articles.values())[:n]

    return most_viewed_articles

### Moduli per le informazioni sulle code

In [4]:
def get_wikipedia_titles(wikidata_ids):
    url = 'https://www.wikidata.org/w/api.php'
    params = {
        'action': 'wbgetentities',
        'format': 'json',
        'ids': '|'.join(wikidata_ids),
        'props': 'sitelinks',
        'sitefilter': 'enwiki'
    }
    response = requests.get(url, params=params)
    data = response.json()

    titles = []
    ent = []
    for entity in data.get('entities', {}).values():
        sitelinks = entity.get('sitelinks', {})
        if 'enwiki' in sitelinks:
            titles.append(sitelinks['enwiki']['title'])
            ent.append(entity['id'])

    df_titles = pd.DataFrame(columns=['titles', 'id'])
    df_titles['titles'] = titles
    df_titles['ent'] = ent
    return df_titles

def get_pageviews(titles):
    url = 'https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/all-agents/{}/daily/20230101/20231231'
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    pageviews = {}

    for title in titles:
        title_formatted = title.replace(' ', '_')
        response = requests.get(url.format(title_formatted), headers={'User-Agent': user_agent})
        if response.status_code == 200:
            data = response.json()
            views = sum(item['views'] for item in data['items'])
            pageviews[title] = views
        else:
            pageviews[title] = "None"

    return pageviews

def process_wikidata_entities(wikidata_ids):
    results = {}
    batch_size = 50

    df = pd.DataFrame(columns=['id', 'title', 'views'])

    for i in tqdm(range(0, len(wikidata_ids), batch_size), "Batch"):
        batch = wikidata_ids[i:i+batch_size]
        #print("Before get_wikipedia_titles: ", time.time() - start_time)
        titles = get_wikipedia_titles(batch)
        #print("After get_wikipedia_titles: ", time.time() - start_time)
        pageviews = get_pageviews(titles.titles.to_list())
        #print("After get_pageviews: ", time.time() - start_time)

        app = pd.DataFrame({'id':titles.ent.to_list(), 'title':titles.titles.to_list(), 'views':pageviews.values()})
        df = pd.concat([df, app], ignore_index=True)
        results.update(pageviews)

    return results, df

## Calcolo paginie più visitate

In [5]:
n = 1000  # Numero di pagine uniche richieste
most_viewed_pages = most_viewed(n)
print(f"Fetched {len(most_viewed_pages)} pages")

Fetched 1000 pages


In [6]:
entities = fetch_wikidata_ids(most_viewed_pages)
entities_id = [str(element['wikidata_id']) for element in list(entities.values())]
entities_views = [str(element['views']) for element in list(entities.values())]
entities_title = entities.keys()

In [7]:
df_ent = pd.DataFrame(columns=['pageTitle', 'views', 'wikidata_id'])
df_ent['pageTitle'] = entities_title
df_ent['views'] = entities_views
df_ent['wikidata_id'] = entities_id

df_ent.head(3)

,pageTitle,views,wikidata_id
0,Cleopatra,158267,Q635
1,Deaths in 2024,151799,Q123489953
2,Dunki (film),209043,Q122341144


## Derivazione delle relazioni dalle pagine più visitate

In [8]:
relations = get_relations_batch(df_ent['wikidata_id'].to_list())
print("Processing complete.")

Processing batch 1 out of 3
Processing batch 2 out of 3
Processing batch 3 out of 3
Processing complete.


### Visualizzazione delle relazioni

In [1]:
for wikidata_id, claims in relations.items():
    print(f"Entity: {wikidata_id}")
    for property_id, claim_list in claims.items():
        print(f"  Property: {property_id}")
        for claim in claim_list:
            mainsnak = claim['mainsnak']
            if 'datavalue' in mainsnak:
                value = mainsnak['datavalue']
                print(f"    Value: {value}")
                break

NameError: name 'relations' is not defined

## Creazione del dataframe

In [ ]:
triples = []
for wikidata_id, claims in relations.items():
    for property_id, claim_list in claims.items():
        for claim in claim_list:
            mainsnak = claim['mainsnak']
            if 'datavalue' in mainsnak:
                value = mainsnak['datavalue']
                if value['type'] == 'wikibase-entityid':
                    value_id = value['value']['id']
                    #print(f"Entity: {wikidata_id} Relation: {property_id} Entity: {value_id}")
                    triples.append({'entity': wikidata_id, 'rel': property_id, 'objt': value_id})

df = pd.DataFrame(triples)
df.head(3)

In [ ]:
print(f"Numero di entità (head) presenti: {df.entity.nunique()}")
print(f"Numero di relazioni presenti: {df.rel.nunique()}")
print(f"Numero di entità (tail) presenti: {df.objt.nunique()}")

In [9]:
df_try = df.merge(df_ent, right_on='wikidata_id', left_on='entity', how='left')
df_try=df_try.drop(['wikidata_id'], axis=1)
df_try=df_try.rename(columns={'pageTitle': 'pageTitle_entity', 'views':'views_entity'})
df_try.head(3)

,entity,rel,objt,pageTitle_entity,views_entity
0,Q635,P22,Q39991,Cleopatra,158267
1,Q635,P21,Q6581072,Cleopatra,158267
2,Q635,P509,Q114953,Cleopatra,158267


In [10]:
df_try.to_csv("./data/df_triple.csv", index=None)

## Riduzione del numero di code

In [8]:
if 'df_try' not in globals():
    df_try = pd.read_csv("./data/df_triple.csv", header=0)
    
df_try.shape

(12312, 5)

### Rimozione delle entità già vestite

In [14]:
df_try_new_entites = df_try[~df_try.objt.isin(df_try.entity)]
df_try_new_entites.shape

(12003, 5)

In [15]:
start_time = time.time()

wikidata_ids = df_try_new_entites['objt'].to_list()
results, df_res_tail = process_wikidata_entities(wikidata_ids)

end_time = time.time()

execution_time = end_time - start_time
print(f"Tempo di esecuzione: {execution_time/60} minuti")

Batch:   1%|          | 2/241 [00:22<44:21, 11.14s/it]


KeyboardInterrupt: 

In [13]:
df_unito = pd.merge(df_try, df_res_tail, left_on='objt', right_on='id', how='left')
df_unito = df_unito.drop(['id'], axis=1).drop_duplicates()
df_unito = df_unito.rename(columns={'title': 'pageTitle_objt', 'views':'views_objt'})

df_unito.head(3)

,entity,rel,objt,pageTitle_entity,views_entity,pageTitle_objt,views_objt
0,Q635,P22,Q39991,Cleopatra,158267,Ptolemy XII Auletes,264658
1,Q635,P21,Q6581072,Cleopatra,158267,Female (gender),2194
9,Q635,P509,Q114953,Cleopatra,158267,NaN,NaN


In [14]:
df_unito.to_csv("./data/df_triple_info.csv", index=None)